<a href="https://colab.research.google.com/github/h1pp-o/Pipeline_Sales_Expanstion_Strategy/blob/main/07_Revenue_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
This File Assumes:
  uses "5. QuickBook.csv" as the data source
  produces Forecast by Different Customer Tiers
'''

In [ ]:
#@title Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Import Modules

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import datetime as dt

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from itertools import product
from typing import Union
from tqdm import notebook
from statsmodels.tsa.seasonal import STL
from datetime import datetime

In [ ]:
#@title Global Path Variables
PATH = '/content/drive/Shareddrives/OPIM 5770/Files to Present To Torigen/Final Data Files'
QUICKBOOKS = '5. QuickBooks.csv'

qb = pd.read_csv(os.path.join(PATH,QUICKBOOKS))

# Process QuickBooks Data

In [ ]:
#@title Step 1: (df) Process raw data
df = (
qb.loc[:, ['Customer full name','Invoice number','Invoice date','Amount',]] \
  .assign(
      Invoice_date=lambda df: pd.to_datetime(df['Invoice date']),
      Year=lambda df: df['Invoice_date'].dt.year,
      Quarter=lambda df: df['Invoice_date'].dt.quarter,
      Amount=lambda df: pd.to_numeric(df['Amount'].str.replace('[\$,]', '', regex=True), errors='coerce')
  ) \
  .drop(columns=['Invoice date']) \
  .query('`Customer full name` not in ["test account", "Company True Name + Record ID"]') \
  .pipe(lambda df: df.assign(**{'Customer full name': df['Customer full name'].replace({'Pet Pals Holistic Veterinary': '12725358701 - Pet Pals Holistic Veterinary Hospital'})})) \
  .groupby(['Customer full name', 'Year', 'Quarter']) \
  .agg(
      Invoice_Count=('Invoice number', 'count'),
      Sales_Revenue=('Amount', 'sum')
  ) \
  .sort_values(['Year', 'Quarter','Customer full name']) \
  .reset_index() \
)

In [ ]:
#@title Clinic Exploration

#Explore Clinics without'numbers-'
#df[~df['Customer full name'].str.contains('-')]

#Explore Clinics with multiple names
# Never mind. It's too much manual work. Assume that we did.

In [ ]:
#@title Step 2: (df2) Join all existing Year-Quarter

# Create time_period and unique_customer dataframes
qb_year = [i for i in range(2020, 2024)]
qb_quarter = [i for i in range(1, 5)]
time_period = pd.DataFrame([(y, q) for y in qb_year for q in qb_quarter if not (y == 2023 and q > 3)], columns=['Year', 'Quarter'])
unique_customer = pd.DataFrame({'Customer full name': df['Customer full name'].unique()})

# Perform cross join and left join using method chaining
df2= (
    unique_customer.assign(key=1)  # Add key for cross join
    .merge(time_period.assign(key=1), on='key')  # Perform cross join with time_period
    .merge(df, on=['Customer full name', 'Year', 'Quarter'], how='left')  # Left join with transaction data
    .fillna({'Invoice_Count': 0, 'Sales_Revenue': 0})  # Fill NA values in specific columns
    .drop(columns=['key'])  # Drop the key column
)

In [ ]:
#@title Test by aggregating Invoice Count & Sales Revenue

print("Sum of Invoice_Count in df:", df['Invoice_Count'].sum())
print("Sum of Invoice_Count in combined_df:", df2['Invoice_Count'].sum())

print("Sum of Sales_Revenue in df:", df['Sales_Revenue'].sum())
print("Sum of Sales_Revenue in combined_df:", df2['Sales_Revenue'].sum())

Sum of Invoice_Count in df: 10679
Sum of Invoice_Count in combined_df: 10679.0
Sum of Sales_Revenue in df: 5487103.12
Sum of Sales_Revenue in combined_df: 5487103.12


In [ ]:
#@title Create function to assign Customer_Tier
def assign_customer_status(rolling_count, has_prior_purchase):
    if rolling_count == 0 and has_prior_purchase == 0:
        return 'Unconverted'
    elif rolling_count == 0 and has_prior_purchase == 1:
        return 'Dormant'
    elif rolling_count == 1:
        return 'One-time'
    elif 2 <= rolling_count <= 4:
        return 'Repeat'
    elif rolling_count >= 5:
        return 'Frequent'

In [ ]:
#@title Determine Moving Window Size for Customer Status

# number of quarters
window_size = 4

In [ ]:
#@title Step 3: (df3) Derive new columns & further process

df3 = (
    df2
    # Convert Year and Quarter to a Datetime Format and Merge Year and Quarter
    .assign(
        Date=lambda x: pd.to_datetime(x['Year'].astype(str) + 'Q' + x['Quarter'].astype(str)),
        Year_Quarter=lambda x: "'" + x['Year'].astype(str).str[-2:] + ' Q' + x['Quarter'].astype(str)
    )
    # Sort DataFrame
    .sort_values(by=['Customer full name', 'Date'])
    # Derive Columns for Cumulative and Rolling Metrics
    .assign(
        Cumulative_Invoice_Count=lambda x: x.groupby('Customer full name')['Invoice_Count'].cumsum(),
        Cumulative_Sales_Revenue=lambda x: x.groupby('Customer full name')['Sales_Revenue'].cumsum(),
        Rolling_Invoice_Count=lambda x: x.groupby('Customer full name')['Invoice_Count']
                                          .rolling(window=window_size, min_periods=1).sum().reset_index(level=0, drop=True),
        Shifted_Cumulative_Invoice_Count=lambda x: x.groupby('Customer full name')['Cumulative_Invoice_Count'].shift(1).fillna(0),
        Has_Prior_Purchase=lambda x: x['Shifted_Cumulative_Invoice_Count'].apply(lambda y: 1 if y > 0 else 0),
    )
    # Applying the function to each row in DataFrame
    .assign(
        Customer_Tier=lambda x: x.apply(lambda row: assign_customer_status(row['Rolling_Invoice_Count'], row['Has_Prior_Purchase']), axis=1)
    )
    # Reorder columns
    [[
        'Customer full name', 'Date', 'Year', 'Quarter', 'Year_Quarter', 'Customer_Tier', 'Invoice_Count',
        'Cumulative_Invoice_Count', 'Rolling_Invoice_Count', 'Shifted_Cumulative_Invoice_Count',
        'Sales_Revenue', 'Cumulative_Sales_Revenue', 'Has_Prior_Purchase'
    ]]
)

In [ ]:
#@title previous version of derive New Columns

# # Convert Year and Quarter to a Datetime Format
# df2['Date'] = pd.to_datetime(df2['Year'].astype(str) + 'Q' + df2['Quarter'].astype(str))

# # Merge Year and Quarter
# df2['Year-Quarter'] = "'" + df2['Year'].astype(str).str[-2:] + ' Q' + df2['Quarter'].astype(str)

# # Sort DataFrame
# df2.sort_values(by=['Customer full name', 'Date'], inplace=True)

# # Derive Columns for Cumulative Metrics
# df2['Cumulative_Invoice_Count'] = df2.groupby('Customer full name')['Invoice_Count'].cumsum()
# df2['Cumulative_Sales_Revenue'] = df2.groupby('Customer full name')['Sales_Revenue'].cumsum()

# # Rolling Invoice Count
# df2['Rolling_Invoice_Count'] = df2.groupby('Customer full name')['Invoice_Count'].rolling(window=window_size, min_periods=1).sum().reset_index(level=0, drop=True)

# # Shifted_Cumulative_Invoice_Count
# df2['Shifted_Cumulative_Invoice_Count'] = df2.groupby('Customer full name')['Cumulative_Invoice_Count'].shift(1)
# df2['Shifted_Cumulative_Invoice_Count'].fillna(0, inplace=True)
# df2['Has_Prior_Purchase'] = df2['Shifted_Cumulative_Invoice_Count'].apply(lambda x: 1 if x > 0 else 0)

# # Applying the function to each row in DataFrame
# df2['Customer_Tier'] = df2.apply(lambda row: assign_customer_status(row['Rolling_Invoice_Count'], row['Has_Prior_Purchase']), axis=1)

# # Reorder columns
# df2 = df2[['Customer full name','Date','Year','Quarter','Year-Quarter','Customer_Tier','Invoice_Count','Cumulative_Invoice_Count','Rolling_Invoice_Count','Shifted_Cumulative_Invoice_Count','Sales_Revenue','Cumulative_Sales_Revenue','Has_Prior_Purchase']]

In [ ]:
#@title Extract Good Sample Clinics

# Calculate the average of Cumulative_Invoice_Count
average_cumulative_count = df3['Cumulative_Invoice_Count'].mean()

# Define a threshold for 'near' (e.g., within 10% of the average)
threshold = 0.06 * average_cumulative_count

# Filter customers whose Cumulative_Invoice_Count is within the threshold of the average
near_average_customers = df2[abs(df3['Cumulative_Invoice_Count'] - average_cumulative_count) <= threshold]['Customer full name'].unique()

#print(near_average_customers)

<ipython-input-12-045bfff6c361>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  near_average_customers = df2[abs(df3['Cumulative_Invoice_Count'] - average_cumulative_count) <= threshold]['Customer full name'].unique()


In [ ]:
#@title Perfect Example

#df3[df3['Customer full name'] == '12779037606 - All Creatures Animal Animal Clinic']

In [ ]:
#@title Step 4: (tier_grouped_data) Group data by Year, Quarter, Customer_Tier

# Grouping data
tier_grouped_data = df3.groupby(['Date','Year','Quarter','Year_Quarter','Customer_Tier']).agg({
    'Customer full name': 'count',
    'Invoice_Count': 'sum',
    'Sales_Revenue': 'sum'
}).reset_index()

# Renaming columns
tier_grouped_data.rename(columns={'Customer full name': 'Customer_Count'}, inplace=True)

In [ ]:
#@title Step 5: (individual dataframe) Create pivoted tables for each measure

def create_pivot_table(data, values_column):
    pivot = data.pivot_table(index='Customer_Tier',
                             columns=['Year', 'Quarter'],
                             values=values_column,
                             fill_value=0)
    return pivot.reindex(['Unconverted', 'Dormant', 'One-time', 'Repeat', 'Frequent'])

# Assuming tier_grouped_data is your DataFrame
df3_custcount = create_pivot_table(tier_grouped_data, 'Customer_Count')
df3_invcount = create_pivot_table(tier_grouped_data, 'Invoice_Count')
df3_salesrev = create_pivot_table(tier_grouped_data, 'Sales_Revenue')

In [ ]:
#@title Prorate remaining Q3

# Most Recent Date ~ Q3 Start
days_elapsed = (pd.to_datetime(qb['Invoice date']).max().date() - pd.to_datetime('2023-07-01').date()).days + 1

# Q3 End ~ Q3 Start
days_total = (pd.to_datetime('2023-09-30').date() - pd.to_datetime('2023-07-01').date()).days + 1

# Replicate df_sum
df3_salesrev_pro = df3_salesrev.copy()

# Using a tuple to reference the multi-level column
q3_2023_col = (2023, 3)

# Prorate for the last Q3
df3_salesrev_pro[q3_2023_col] = round(df3_salesrev[q3_2023_col] / days_elapsed * days_total, 2)

In [ ]:
#@title Test each table

df3_custcount
#df3_invcount
#df3_salesrev # no longer needed
#df3_salesrev_pro

Year           2020                    2021                 2022            \
Quarter           1     2     3     4     1     2    3    4    1    2    3   
Customer_Tier                                                                
Unconverted    1325  1304  1252  1202  1134  1022  911  806  689  613  530   
Dormant           0     0     0     0    30    43   69   94  125  163  228   
One-time         27    34    65    91   100   134  138  152  165  182  204   
Repeat           25    29    48    58    75   108  155  186  235  238  237   
Frequent          4    14    16    30    42    74  108  143  167  185  182   

Year               2023            
Quarter          4    1    2    3  
Customer_Tier                      
Unconverted    427  313  135    0  
Dormant        310  405  463  521  
One-time       189  208  245  291  
Repeat         270  268  314  326  
Frequent       185  187  224  243

# Forecast

## Preparation

In [ ]:
#@title Step 6: (df4) Further reshape for forecasting

def reshape_and_reorder(data, value_column, order_columns):
    # Pivot the data
    reshaped_data = (data[['Date', 'Customer_Tier', value_column]]
                     .pivot(index='Date', columns='Customer_Tier', values=value_column)
                     .fillna(0)
                     .assign(Is_Forecast=0))

    # Reorder the columns as per the provided list
    reordered_data = reshaped_data[order_columns]

    return reordered_data

# Define the column order
columns_order = ['Unconverted', 'Dormant', 'One-time', 'Repeat', 'Frequent', 'Is_Forecast']

# Applying the function to each DataFrame
df4_custcount = reshape_and_reorder(tier_grouped_data, 'Customer_Count', columns_order)
df4_invcount = reshape_and_reorder(tier_grouped_data, 'Invoice_Count', columns_order)
df4_salesrev_pro = reshape_and_reorder(tier_grouped_data, 'Sales_Revenue', columns_order)

In [ ]:
#@title Define function for ARIMA Model

ps = range(0, 4, 1)
qs = range(0, 4, 1)

order_list = list(product(ps, qs))


def optimize_ARMA(endog: Union[pd.Series, list], order_list: list) -> pd.DataFrame:
    results = []

    for order in notebook.tqdm(order_list):
        try:
            model = SARIMAX(endog, order=(order[0], 0, order[1]),
 simple_differencing=False).fit(disp=False)
        except:
            continue

        aic = model.aic
        results.append([order, aic])

    result_df = pd.DataFrame(results)
    result_df.columns = ['(p,q)', 'AIC']

    #Sort in ascending order, lower AIC is better
    result_df = result_df.sort_values(by='AIC',
 ascending=True).reset_index(drop=True)

    return result_df

## Optimize Model Parameters

In [ ]:
#df3_custcount
#df3_invcount
#df4_salesrev_pro

In [ ]:
#df4_custcount
#df4_invcount
#df4_salesrev_pro

In [ ]:
#@title Dormant: Customer Count (p=2, q=0)
#optimize_ARMA(df4_custcount['Dormant'], order_list)

In [ ]:
#@title One-time: Customer Count (p=2, q=0)
#optimize_ARMA(df4_custcount['One-time'], order_list)

In [ ]:
#@title One-time: Sales Revenue (p=1, q=0)
#optimize_ARMA(df4_salesrev_pro['One-time'], order_list)

In [ ]:
#@title Repeat: Customer Count (p=1, q=2)
#optimize_ARMA(df4_custcount['Repeat'], order_list)

In [ ]:
#@title Repeat: Sales Revenue (p=2, q=0)
#optimize_ARMA(df4_salesrev_pro['Repeat'], order_list)

In [ ]:
#@title Frequent: Customer Count (p=1, q=2)
#optimize_ARMA(df4_custcount['Frequent'], order_list)

In [ ]:
#@title Frequent: Sales Revenue (p=3, q=0)
#optimize_ARMA(df4_salesrev_pro['Frequent'], order_list)

## Fit Model

In [ ]:
#@title Dormant: Customer Count

model = sm.tsa.statespace.SARIMAX(df4_custcount['Dormant'],
                                  order=(2, 0, 0),
                                  seasonal_order=(0, 0, 0, 4),
                                  enforce_stationarity=False,
                                  enforce_invertibility=False)

results = model.fit()

# Forecast the next 5 quarters
forecast_dormant_custcount = results.get_forecast(steps=5)

# Set Variable
pred_dormant_custcount = forecast_dormant_custcount.predicted_mean

# Set Confidence Interval
CI_dormant_custcount = forecast_dormant_custcount.conf_int()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)


In [ ]:
#@title One-time: Customer Count

model = sm.tsa.statespace.SARIMAX(df4_custcount['One-time'],
                                  order=(2, 0, 0),
                                  seasonal_order=(0, 0, 0, 4),
                                  enforce_stationarity=False,
                                  enforce_invertibility=False)

results = model.fit()

# Forecast the next 5 quarters
forecast_onetime_custcount = results.get_forecast(steps=5)

# Set Variable
pred_onetime_custcount = forecast_onetime_custcount.predicted_mean

# Set Confidence Interval
CI_onetime_custcount = forecast_onetime_custcount.conf_int()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)


In [ ]:
#@title One-time: Sales Revenue

model = sm.tsa.statespace.SARIMAX(df4_salesrev_pro['One-time'],
                                  order=(1, 0, 0),
                                  seasonal_order=(0, 0, 0, 4),
                                  enforce_stationarity=False,
                                  enforce_invertibility=False)

results = model.fit()

# Forecast the next 5 quarters
forecast_onetime_salesrev = results.get_forecast(steps=5)

# Set Variable
pred_onetime_salesrev = forecast_onetime_salesrev.predicted_mean

# Set Confidence Interval
CI_onetime_salesrev = forecast_onetime_salesrev.conf_int()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)


In [ ]:
#@title Repeat Customer Count

model = sm.tsa.statespace.SARIMAX(df4_custcount['One-time'],
                                  order=(1, 0, 2),
                                  seasonal_order=(0, 0, 0, 4),
                                  enforce_stationarity=False,
                                  enforce_invertibility=False)

results = model.fit()

# Forecast the next 5 quarters
forecast_repeat_custcount = results.get_forecast(steps=5)

# Set Variable
pred_repeat_custcount = forecast_repeat_custcount.predicted_mean

# Set Confidence Interval
CI_repeat_custcount = forecast_repeat_custcount.conf_int()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)


In [ ]:
#@title Repeat: Sales Revenue

model = sm.tsa.statespace.SARIMAX(df4_salesrev_pro['Repeat'],
                                  order=(2, 0, 0),
                                  seasonal_order=(0, 0, 0, 4)
)

results = model.fit()

# Forecast the next 4 quarters
forecast_repeat_salesrev = results.get_forecast(steps=5)

# Set Variable
pred_repeat_salesrev = forecast_repeat_salesrev.predicted_mean

# Set Confidence Interval
CI_repeat_salesrev = forecast_repeat_salesrev.conf_int()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


In [ ]:
#@title Clinic Count for Frequent

model = sm.tsa.statespace.SARIMAX(df4_custcount['Frequent'],
                                  order=(1, 0, 2),
                                  seasonal_order=(0, 0, 0, 4)
                                  )

results = model.fit()

# Forecast the next 4 quarters
forecast_frequent_custcount = results.get_forecast(steps=5)

# Set Variable
pred_frequent_custcount = forecast_frequent_custcount.predicted_mean

# Set Confidence Interval
CI_frequent_custcount = forecast_frequent_custcount.conf_int()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


In [ ]:
#@title Sales Revenue for Frequent

model = sm.tsa.statespace.SARIMAX(df4_salesrev_pro['Frequent'],
                                  order=(3, 0, 0),
                                  seasonal_order=(0, 0, 0, 4)
                                  )

results = model.fit()

# Forecast the next 4 quarters
forecast_frequent_salesrev = results.get_forecast(steps=5)

# Set Variable
pred_frequent_salesrev = forecast_frequent_salesrev.predicted_mean

# Set Confidence Interval
CI_frequent_salesrev = forecast_frequent_salesrev.conf_int()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


## Step 7: Create Dataframe for Forecasted Time Periods

In [ ]:
### Dormant
#pred_dormant_custcount
#CI_dormant_custcount

### One-time
#pred_onetime_custcount
#CI_onetime_custcount
#pred_onetime_salesrev
#CI_onetime_salesrev

### Repeat
#pred_repeat_custcount
#CI_repeat_custcount
#pred_repeat_salesrev
#CI_repeat_salesrev

### Frequent
#pred_frequent_custcount
#CI_frequent_custcount
#pred_frequent_salesrev
#CI_frequent_salesrev

In [ ]:
#df4_custcount
#df4_invcount
#df4_salesrev_pro

Customer_Tier,Unconverted,Dormant,One-time,Repeat,Frequent,Is_Forecast
Date,,,,,,
2020-01-01,0.0,0.0,16068.00,50026.76,14588.49,0
2020-04-01,0.0,0.0,11615.00,18560.00,57950.78,0
2020-07-01,0.0,0.0,25868.99,39262.00,64085.00,0
2020-10-01,0.0,0.0,23075.00,43646.00,130709.49,0
2021-01-01,0.0,0.0,23279.00,87476.00,160545.95,0
2021-04-01,0.0,0.0,25626.00,96805.00,209908.00,0
2021-07-01,0.0,0.0,30639.50,135325.98,277096.32,0
2021-10-01,0.0,0.0,44271.00,91937.90,308082.39,0
2022-01-01,0.0,0.0,37807.00,152404.00,290043.77,0


In [ ]:
#@title Customer Count forecast

dates = ['2023-10-01', '2024-01-01', '2024-04-01', '2024-07-01', '2024-10-01']
is_forecast = [1] * len(dates)
unconverted = [0.0] * len(dates)

# Create the DataFrame in one step
df4_custcount_pred = pd.DataFrame({
    'Date': dates,
    'Unconverted': unconverted,
    'Dormant': pred_dormant_custcount,
    'One-time': pred_onetime_custcount,
    'Repeat': pred_repeat_custcount,
    'Frequent': pred_frequent_custcount,
    'Is_Forecast': is_forecast
}).set_index('Date')


In [ ]:
#@title Sales Revenue forecast

dates = ['2023-10-01', '2024-01-01', '2024-04-01', '2024-07-01', '2024-10-01']
is_forecast = [1] * len(dates)
unconverted = [0.0] * len(dates)

# Create the DataFrame in one step
df4_salesrev_pred = pd.DataFrame({
    'Date': dates,
    'Unconverted': 0.0,
    'Dormant': 0.0,
    'One-time': pred_onetime_salesrev,
    'Repeat': pred_repeat_salesrev,
    'Frequent': pred_frequent_salesrev,
    'Is_Forecast': is_forecast
}).set_index('Date')

In [ ]:
#df4_custcount
#df4_custcount_pred

#df4_salesrev_pro
df4_salesrev_pred

,Unconverted,Dormant,One-time,Repeat,Frequent,Is_Forecast
Date,,,,,,
2023-10-01,0.0,0.0,28834.607676,151562.198170,430224.991973,1
2024-01-01,0.0,0.0,28806.243281,113860.752858,428591.307646,1
2024-04-01,0.0,0.0,28777.906788,137783.969069,443471.163055,1
2024-07-01,0.0,0.0,28749.598169,117580.295433,436695.678025,1
2024-10-01,0.0,0.0,28721.317398,129047.578098,437753.015374,1


## Step 8: Reshape data for BI tool consumption

In [ ]:
#@title Reshape Customer Count data

df5_custcount =  (df4_custcount \
                    .reset_index() \
                    .pipe(lambda x: pd.concat([x, df4_custcount_pred.reset_index()], ignore_index=True)) \
                    .assign(**{'Unconverted': lambda x: x['Unconverted'].round(0).astype(int),
                               'Dormant': lambda x: x['Dormant'].round(0).astype(int),
                               'One-time': lambda x: x['One-time'].round(0).astype(int),
                               'Repeat': lambda x: x['Repeat'].round(0).astype(int),
                               'Frequent': lambda x: x['Frequent'].round(0).astype(int),
                               'Date': lambda x: pd.to_datetime(x['Date']).dt.date}) \
                    .melt(id_vars=['Date', 'Is_Forecast'],
                          value_vars=['Unconverted','Dormant','One-time', 'Repeat', 'Frequent'],
                          var_name='Customer_Tier',
                          value_name='Customer_Count')
                    )

In [ ]:
#@title Reshape Sales Revenue data

df5_salesrev =  (df4_salesrev_pro \
                    .reset_index() \
                    .pipe(lambda x: pd.concat([x, df4_salesrev_pred.reset_index()], ignore_index=True)) \
                    .assign(**{'Unconverted': lambda x: x['Unconverted'].round(2).astype(float),
                               'Dormant': lambda x: x['Dormant'].round(2).astype(float),
                               'One-time': lambda x: x['One-time'].round(2).astype(float),
                               'Repeat': lambda x: x['Repeat'].round(2).astype(float),
                               'Frequent': lambda x: x['Frequent'].round(2).astype(float),
                               'Date': lambda x: pd.to_datetime(x['Date']).dt.date}) \
                    .melt(id_vars=['Date', 'Is_Forecast'],
                          value_vars=['Unconverted','Dormant','One-time', 'Repeat', 'Frequent'],
                          var_name='Customer_Tier',
                          value_name='Sales_Revenue')
                    )

In [ ]:
#df5_custcount
#df5_salesrev

,Date,Is_Forecast,Customer_Tier,Customer_Count
0,2020-01-01,0,Unconverted,1325
1,2020-04-01,0,Unconverted,1304
2,2020-07-01,0,Unconverted,1252
3,2020-10-01,0,Unconverted,1202
4,2021-01-01,0,Unconverted,1134
...,...,...,...,...
95,2023-10-01,1,Frequent,257
96,2024-01-01,1,Frequent,247
97,2024-04-01,1,Frequent,243
98,2024-07-01,1,Frequent,239


In [ ]:
# For df5_custcount
df6_custcount = df5_custcount.assign(
    Customer_Index=np.where(df5_custcount['Customer_Tier'] == 'Unconverted', 1,
                   np.where(df5_custcount['Customer_Tier'] == 'Dormant', 2,
                   np.where(df5_custcount['Customer_Tier'] == 'One-time', 3,
                   np.where(df5_custcount['Customer_Tier'] == 'Repeat', 4,
                   np.where(df5_custcount['Customer_Tier'] == 'Frequent', 5, 0))))),
    Date_Index=pd.to_datetime(df5_custcount['Date']).dt.strftime('%Y%m%d'),
    Forecast_Key=lambda x: x['Date_Index'].astype(str) + x['Customer_Index'].astype(str)
)

# For df5_custcount
df6_salesrev = df5_salesrev.assign(
    Customer_Index=np.where(df5_custcount['Customer_Tier'] == 'Unconverted', 1,
                   np.where(df5_custcount['Customer_Tier'] == 'Dormant', 2,
                   np.where(df5_custcount['Customer_Tier'] == 'One-time', 3,
                   np.where(df5_custcount['Customer_Tier'] == 'Repeat', 4,
                   np.where(df5_custcount['Customer_Tier'] == 'Frequent', 5, 0))))),
    Date_Index=pd.to_datetime(df5_custcount['Date']).dt.strftime('%Y%m%d'),
    Forecast_Key=lambda x: x['Date_Index'].astype(str) + x['Customer_Index'].astype(str)
)

# Merging df_sum and df_count
df_merge = pd.merge(df6_salesrev, df6_custcount, on='Forecast_Key', how='left')

# Cleaning and further processing the merged DataFrame
df_merge = df_merge.drop(
    [col for col in df_merge.columns if '_y' in col], axis=1
).rename(columns=lambda x: x.replace('_x', '')).assign(
    Clinic_Count=lambda x: x['Customer_Count'].fillna(0.0),
    Cumulative_Customer_Count=lambda x: x.groupby('Customer_Tier')['Customer_Count'].cumsum(),
    Cumulative_Sales_Revenue=lambda x: x.groupby('Customer_Tier')['Sales_Revenue'].cumsum()
).assign(
    Vaccine_per_Clinic=lambda x: np.where(x['Cumulative_Customer_Count'] != 0,
                                          x['Sales_Revenue'] / 1000 / x['Customer_Count'], 0) # recheck logic
)

# Dropping columns if they exist
columns_to_drop = ['Customer_Index', 'Forecast_Key']
df_merge = df_merge.drop(columns=[col for col in columns_to_drop if col in df_merge.columns])

# Reorder columns
df_merge = df_merge[['Date', 'Date_Index', 'Is_Forecast', 'Customer_Tier',
                     'Customer_Count', 'Cumulative_Customer_Count', 'Sales_Revenue',
                     'Cumulative_Sales_Revenue', 'Vaccine_per_Clinic']]

# BOOKMARK

In [ ]:
#@title Test Variables

#df6_custcount
#df6_salesrev
#df_merge

In [ ]:
df_merge[df_merge['Customer_Tier'] == 'Frequent']

,Date,Date_Index,Is_Forecast,Customer_Tier,Customer_Count,Cumulative_Customer_Count,Sales_Revenue,Cumulative_Sales_Revenue,Vaccine_per_Clinic
80,2020-01-01,20200101,0,Frequent,4,4,14588.49,14588.49,3.647123
81,2020-04-01,20200401,0,Frequent,14,18,57950.78,72539.27,4.139341
82,2020-07-01,20200701,0,Frequent,16,34,64085.00,136624.27,4.005312
83,2020-10-01,20201001,0,Frequent,30,64,130709.49,267333.76,4.356983
84,2021-01-01,20210101,0,Frequent,42,106,160545.95,427879.71,3.822523
85,2021-04-01,20210401,0,Frequent,74,180,209908.00,637787.71,2.836595
86,2021-07-01,20210701,0,Frequent,108,288,277096.32,914884.03,2.565707
87,2021-10-01,20211001,0,Frequent,143,431,308082.39,1222966.42,2.154422
88,2022-01-01,20220101,0,Frequent,167,598,290043.77,1513010.19,1.736789
89,2022-04-01,20220401,0,Frequent,185,783,335830.32,1848840.51,1.815299


# Export Data

In [ ]:
#@title Export data as csv
#df_merge.to_csv(os.path.join(PATH,'15. Forecast.csv'), index=False)

In [ ]:
#@title Summary of variables

#qb
#df #intermediate
#df2 #intermediate
#df3

#tier_grouped_data
#df3_custcount
#df3_salesrev_pro

#df4_custcount #forecast
#df4_salesrev_pro #forecast

#df5_custcount #melt #intermediate
#df5_salesrev #melt #intermediate
#df6_custcount
#df6_salesrev

#df_merge